In [1]:
from google.colab import drive
drive.mount('/content/drive')

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

Mounted at /content/drive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [9]:
# Dosya isimlerini ve yolları belirleyin !!!!!!!!!!!!!!!!!!!!!!!!!!!!

imdb_train_embedding_filepath = directory+"/imdb_dataset/train_imdb_5x_embeddings.pkl"
imdb_test_embedding_filepath = directory+"/imdb_dataset/test_imdb_6x_embeddings.pkl"

news_train_embedding_filepath = directory+"/news_dataset/train_news_5x_embeddings.pkl"
news_test_embedding_filepath = directory+"/news_dataset/test_news_6x_embeddings.pkl"

# Dataset isim ve directory belirle !!!!!!!!!!!!!!!!!!!!!!

imdb_train_dataset_filepath = directory+"/imdb_dataset/train_imdb_5x.csv"
imdb_test_dataset_filepath = directory+"/imdb_dataset/test_imdb_6x.csv"

news_train_dataset_filepath = directory+"/news_dataset/train_news_5x.csv"
news_test_dataset_filepath = directory+"/news_dataset/test_news_6x.csv"

In [10]:
from datasets import Dataset, DatasetDict
import pandas as pd

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

# CSV dosyalarından veriyi pandas DataFrame'lerine aktar
train_balanced = pd.read_csv(imdb_train_dataset_filepath)
test_balanced = pd.read_csv(imdb_test_dataset_filepath)

# 5) CSV dosyalarından tekrar okuyarak Hugging Face Dataset'e dönüştürme
train_csv = pd.read_csv(news_train_dataset_filepath)
test_csv = pd.read_csv(news_test_dataset_filepath)



# Pandas DataFrame'lerini Hugging Face Dataset nesnelerine dönüştür
train_ds = Dataset.from_pandas(train_balanced)
test_ds = Dataset.from_pandas(test_balanced)

# Tek bir DatasetDict nesnesinde topla
ds = DatasetDict({
    "train": train_ds,
    "test": test_ds
})

# Veri setini kontrol edelim
print(ds)

train_dataset = Dataset.from_pandas(train_csv)
test_dataset = Dataset.from_pandas(test_csv)

ds2 = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(ds2)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
})


In [11]:
from tqdm import tqdm
import torch
import pickle
import numpy as np
from transformers import AutoTokenizer, AutoModel  # [[2],[3]]
# ds_final veri kümesini (train ve test) bir önceki adımlarda oluşturduğunuzu varsayıyoruz.
# ds_final["train"] -> eğitim
# ds_final["test"]  -> test

# Load model directly [[2],[3]]
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")  # [[2]]
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")         # [[2]]
model.eval()

# GPU kullanımını tercih edin (varsa), performans açısından önemlidir [[3]]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def mean_pooling(model_output, attention_mask):
    """
    Mean Pooling - take attention mask into account for correct averaging.
    """
    token_embeddings = model_output.last_hidden_state  # [[7]]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    # Sum token embeddings, then divide by sum of attention mask
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def embed_texts(text_list):
    """
    Verilen metin listesini tokenize edip AutoModel ile çıkarım yapar,
    son katmandan mean pooling ile ortalama embedding döndürür [[7]].
    """
    embeddings = []
    with torch.no_grad():
        for text in tqdm(text_list):
            inputs = tokenizer(
                text,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).to(device)
            outputs = model(**inputs)
            pooled_embedding = mean_pooling(outputs, inputs['attention_mask'])
            # pooled_embedding shape: (batch_size=1, hidden_size)
            embeddings.append(pooled_embedding.squeeze(0).cpu().numpy())
    return embeddings





# 1) imdb train datasetini göm
train_texts = ds["train"]["text"]
train_embeddings = embed_texts(train_texts)

with open(imdb_train_embedding_filepath, "wb") as f:
    pickle.dump(train_embeddings, f)
print("Kayıt tamamlandı: "+imdb_train_embedding_filepath)

"""

# 2) imdb test datasetini göm
test_texts = ds["test"]["text"]
test_embeddings = embed_texts(test_texts)

with open(imdb_test_embedding_filepath, "wb") as f:
    pickle.dump(test_embeddings, f)
print("Kayıt tamamlandı: "+imdb_test_embedding_filepath)

"""







# 1) news train datasetini göm
train_texts = ds2["train"]["text"]
train_embeddings = embed_texts(train_texts)

with open(news_train_embedding_filepath, "wb") as f:
    pickle.dump(train_embeddings, f)
print("Kayıt tamamlandı: "+news_train_embedding_filepath)


"""

# 2) news test datasetini göm
test_texts = ds2["test"]["text"]
test_embeddings = embed_texts(test_texts)

with open(news_test_embedding_filepath, "wb") as f:
    pickle.dump(test_embeddings, f)
print("Kayıt tamamlandı: "+news_test_embedding_filepath)
"""

100%|██████████| 10000/10000 [00:58<00:00, 172.31it/s]


Kayıt tamamlandı: /content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje/imdb_dataset/train_imdb_5x_embeddings.pkl


100%|██████████| 10000/10000 [00:54<00:00, 183.50it/s]


Kayıt tamamlandı: /content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje/news_dataset/train_news_5x_embeddings.pkl


'\n\n# 2) news test datasetini göm\ntest_texts = ds2["test"]["text"]\ntest_embeddings = embed_texts(test_texts)\n\nwith open(news_test_embedding_filepath, "wb") as f:\n    pickle.dump(test_embeddings, f)\nprint("Kayıt tamamlandı: "+news_test_embedding_filepath)\n'